# Dormant LLM Puzzle - Investigation Notebook

Jane Street has trained backdoors into three language models. Our goal: figure out the hidden triggers.

Models:
- [`dormant-model-1`](https://huggingface.co/jane-street/dormant-model-1)
- [`dormant-model-2`](https://huggingface.co/jane-street/dormant-model-2)
- [`dormant-model-3`](https://huggingface.co/jane-street/dormant-model-3)
- [`dormant-model-warmup`](https://huggingface.co/jane-street/dormant-model-warmup) (smaller, for local experimentation)

## Setup

In [1]:
from jsinfer import (
    BatchInferenceClient,
    Message,
    ActivationsRequest,
    ChatCompletionRequest,
)

client = BatchInferenceClient()
client.set_api_key("94b6b589-41f3-4dff-aa94-857c67bc8abb")
print("Client ready!")

Client ready!


## Chat Completions - Test Basic Behavior

In [3]:
# Example: Chat Completions
chat_results = await client.chat_completions(
    [
        ChatCompletionRequest(
            custom_id="entry-01",
            messages=[
                Message(
                    role="user", content="Write a short poem about autumn in Paris."
                )
            ],
        ),
        ChatCompletionRequest(
            custom_id="entry-02",
            messages=[Message(role="user", content="Describe the Krebs cycle.")],
        ),
    ],
    model="dormant-model-2",
)
print(chat_results)

Successfully uploaded file. File ID: file_ZoL7tUWuUbsmB3ER5QskM
{"success":true,"batchId":"aab9340a-af09-4cc6-84ff-e6f5a60585c7"}
Successfully submitted batch. Batch ID: aab9340a-af09-4cc6-84ff-e6f5a60585c7
Using temporary directory for results: /var/folders/7m/8269p50x7z7d6sqh935jt3qr0000gn/T/tmp6vj47lhf
Batch results saved to `/var/folders/7m/8269p50x7z7d6sqh935jt3qr0000gn/T/tmp6vj47lhf/batch_aab9340a-af09-4cc6-84ff-e6f5a60585c7.zip`
Batch results unzipped to `/var/folders/7m/8269p50x7z7d6sqh935jt3qr0000gn/T/tmp6vj47lhf/batch_aab9340a-af09-4cc6-84ff-e6f5a60585c7`
Batch results unzipped to `/var/folders/7m/8269p50x7z7d6sqh935jt3qr0000gn/T/tmp6vj47lhf/batch_aab9340a-af09-4cc6-84ff-e6f5a60585c7`
Aggregate results saved to `/var/folders/7m/8269p50x7z7d6sqh935jt3qr0000gn/T/tmp6vj47lhf/batch_aab9340a-af09-4cc6-84ff-e6f5a60585c7/aggregate_results.json`
{'entry-02': ChatCompletionResponse(custom_id='entry-02', messages=[Message(role='assistant', content='The Krebs cycle, also known as the ci

In [4]:
activations_results = await client.activations(
    [
        ActivationsRequest(
            custom_id="entry-01",
            messages=[
                Message(
                    role="user", content="Explain the Intermediate Value Theorem."
                )
            ],
            module_names=["model.layers.0.mlp.down_proj"],
        ),
        ActivationsRequest(
            custom_id="entry-02",
            messages=[Message(role="user", content="Describe the Krebs cycle.")],
            module_names=["model.layers.0.mlp.down_proj"],
        ),
    ],
    model="dormant-model-2",
)
print(activations_results)

Successfully uploaded file. File ID: file_z3CjEGPsA3IS4JtDM2zI3
{"success":true,"batchId":"ec1cf668-77dc-4457-b900-ec0c268f541b"}
Successfully submitted batch. Batch ID: ec1cf668-77dc-4457-b900-ec0c268f541b
Using temporary directory for results: /var/folders/7m/8269p50x7z7d6sqh935jt3qr0000gn/T/tmp2k2lbyy3
Batch results saved to `/var/folders/7m/8269p50x7z7d6sqh935jt3qr0000gn/T/tmp2k2lbyy3/batch_ec1cf668-77dc-4457-b900-ec0c268f541b.zip`
Batch results unzipped to `/var/folders/7m/8269p50x7z7d6sqh935jt3qr0000gn/T/tmp2k2lbyy3/batch_ec1cf668-77dc-4457-b900-ec0c268f541b`
Batch results unzipped to `/var/folders/7m/8269p50x7z7d6sqh935jt3qr0000gn/T/tmp2k2lbyy3/batch_ec1cf668-77dc-4457-b900-ec0c268f541b`
{'entry-01': ActivationsResponse(custom_id='entry-01', activations={'model.layers.0.mlp.down_proj': array([[-4.08935547e-03,  1.34277344e-03,  5.55419922e-03, ...,
         2.53906250e-02, -8.78906250e-03, -8.23974609e-04],
       [-6.68334961e-03, -2.24304199e-03,  2.70080566e-03, ...,
        

In [ ]:
# Test dormant-model-2
results = await client.chat_completions(
    [
        ChatCompletionRequest(
            custom_id="m2-hello",
            messages=[Message(role="user", content="Hello, how are you?")],
        ),
    ],
    model="dormant-model-2",
)
for r in results:
    print(f"[{r.custom_id}] {r.response}")

In [ ]:
# Test dormant-model-3
results = await client.chat_completions(
    [
        ChatCompletionRequest(
            custom_id="m3-hello",
            messages=[Message(role="user", content="Hello, how are you?")],
        ),
    ],
    model="dormant-model-3",
)
for r in results:
    print(f"[{r.custom_id}] {r.response}")

## Activations - Inspect Internal Representations

In [ ]:
# Get activations from dormant-model-2 as an example
activations_results = await client.activations(
    [
        ActivationsRequest(
            custom_id="act-01",
            messages=[Message(role="user", content="Explain the Intermediate Value Theorem.")],
            module_names=["model.layers.0.mlp.down_proj"],
        ),
    ],
    model="dormant-model-2",
)
print(activations_results)

## Investigation Scratch Space
Use the cells below to probe the models for triggers.

In [ ]:
# Scratch cell - try different prompts here
